In [1]:
from google.colab import files

# Install Kaggle library
!pip install -q kaggle
u = files.upload()
# upload ur kaggle api

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json
# kaggle dataset
!kaggle datasets download -d smid80/coronavirus-covid19-tweets-early-april

Saving kaggle.json to kaggle.json
100% 1.30G/1.30G [00:32<00:00, 52.1MB/s]
100% 1.30G/1.30G [00:32<00:00, 42.9MB/s]


In [2]:
!pip install tweet-preprocessor

In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re #regular expressions
import string 

from nltk.corpus import stopwords
from nltk import word_tokenize
import preprocessor as p
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

reuse = False

import os
for dirname, _, filenames in os.walk('/content'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/content/coronavirus-covid19-tweets-early-april.zip
/content/kaggle.json
/content/.config/.last_opt_in_prompt.yaml
/content/.config/config_sentinel
/content/.config/gce
/content/.config/.last_update_check.json
/content/.config/active_config
/content/.config/.last_survey_prompt.yaml
/content/.config/.metricsUUID
/content/.config/logs/2020.05.13/16.28.34.606443.log
/content/.config/logs/2020.05.13/16.28.53.120787.log
/content/.config/logs/2020.05.13/16.29.24.028929.log
/content/.config/logs/2020.05.13/16.29.23.578572.log
/content/.config/logs/2020.05.13/16.29.05.704397.log
/content/.config/logs/2020.05.13/16.29.09.991403.log
/content/.config/configurations/config_default
/content/sample_data/README.md
/content/sample_data/anscombe.json
/content/sample_data/mnist_test.csv
/content/sample_data/california_housing_train.csv
/content/sample_data/california_housing_test.csv
/content/sample_data/mnist_train_small.csv


In [0]:
if not os.path.exists('data'):
    os.makedirs('data')


In [0]:
!mv coronavirus-covid19-tweets-early-april.zip data

In [14]:
!unzip '/content/data/coronavirus-covid19-tweets-early-april.zip' -d '/content/data'

Archive:  /content/data/coronavirus-covid19-tweets-early-april.zip
  inflating: /content/data/2020-03-29 Coronavirus Tweets.CSV  
  inflating: /content/data/2020-03-30 Coronavirus Tweets.CSV  
  inflating: /content/data/2020-03-31 Coronavirus Tweets.CSV  
  inflating: /content/data/2020-04-01 Coronavirus Tweets.CSV  
  inflating: /content/data/2020-04-02 Coronavirus Tweets.CSV  
  inflating: /content/data/2020-04-03 Coronavirus Tweets.CSV  
  inflating: /content/data/2020-04-04 Coronavirus Tweets.CSV  
  inflating: /content/data/2020-04-05 Coronavirus Tweets.CSV  
  inflating: /content/data/2020-04-06 Coronavirus Tweets.CSV  
  inflating: /content/data/2020-04-07 Coronavirus Tweets.CSV  
  inflating: /content/data/2020-04-08 Coronavirus Tweets.CSV  
  inflating: /content/data/2020-04-09 Coronavirus Tweets.CSV  
  inflating: /content/data/2020-04-10 Coronavirus Tweets.CSV  
  inflating: /content/data/2020-04-11 Coronavirus Tweets.CSV  
  inflating: /content/data/2020-04-12 Coronavirus T

In [0]:
def join_df(path):
    data = pd.DataFrame()
    sub_path = 'data'
    print(f"In path : {path}{sub_path}")
    for file in os.listdir(path+sub_path):
      if file == "coronavirus-covid19-tweets-early-april.zip":
          continue
      else:
          temp = pd.read_csv(path+sub_path+'/'+file)
          temp = temp[(temp.country_code=='IN') & (temp.lang=='en') ]
          data = pd.concat([data,temp])
    return data
        

In [55]:
path = '/content/'


print('Creating main dataset')
df = join_df(path)
df.to_csv("/content/main.csv")
    
    
print(df.head())
print(df.shape)

Creating main dataset
In path : /content/data
                 status_id             user_id  ... verified lang
292    1245863655586979840  857681810830094338  ...    False   en
6704   1245869432938754048            70794773  ...    False   en
7008   1245869720206639105  857681810830094338  ...    False   en
7736   1245870439231963136            61502443  ...    False   en
13351  1245875580014223360  825342313144651776  ...    False   en

[5 rows x 22 columns]
(23949, 22)


In [56]:
# checking for missing values
df.isnull().sum()

status_id                   0
user_id                     0
created_at                  0
screen_name                 0
text                        0
source                      0
reply_to_status_id      21039
reply_to_user_id        19812
reply_to_screen_name    19812
is_quote                    0
is_retweet                  0
favourites_count            0
retweet_count               0
country_code                0
place_full_name             0
place_type                  0
followers_count             0
friends_count               0
account_lang            23949
account_created_at          0
verified                    0
lang                        0
dtype: int64

In [58]:
#drop colunmns with null values and non relevant columns 

df.drop(df.columns[df.isnull().any()],axis=1,inplace=True)
if reuse:
    
    df.drop(['Unnamed: 0','favourites_count','retweet_count','country_code','followers_count','friends_count','verified','account_created_at','lang'],axis=1,inplace=True)
else:
    df.drop(['favourites_count','retweet_count','country_code','followers_count','friends_count','verified','account_created_at','lang'],axis=1,inplace=True)

    
df.head()

KeyError: ignored

In [59]:
df.place_full_name.value_counts()

Mumbai, India             3030
New Delhi, India          2842
Bengaluru South, India     659
India                      591
Kolkata, India             541
                          ... 
Nabadwip, India              1
Ambur                        1
Raybag, India                1
Haveli                       1
Isnapur, India               1
Name: place_full_name, Length: 1994, dtype: int64

In [60]:
# processing timestamp

datetime = df['created_at'].str.split(pat='T')
datetime = list(map(lambda x: " ".join(x),datetime))
df['datetime'] = pd.to_datetime(datetime)
df.drop('created_at',axis=1,inplace=True)
df['datetime']

292      2020-04-03 00:00:17+00:00
6704     2020-04-03 00:23:14+00:00
7008     2020-04-03 00:24:23+00:00
7736     2020-04-03 00:27:14+00:00
13351    2020-04-03 00:47:40+00:00
                    ...           
414315   2020-04-10 22:57:45+00:00
423093   2020-04-10 23:36:38+00:00
424383   2020-04-10 23:43:12+00:00
426584   2020-04-10 23:54:47+00:00
426929   2020-04-10 23:56:31+00:00
Name: datetime, Length: 23949, dtype: datetime64[ns, UTC]

In [0]:
#cleaning the tweets function

#HappyEmoticons
emoticons_happy = set([
    ':-)', ':)', ';)', ':o)', ':]', ':3', ':c)', ':>', '=]', '8)', '=)', ':}',
    ':^)', ':-D', ':D', '8-D', '8D', 'x-D', 'xD', 'X-D', 'XD', '=-D', '=D',
    '=-3', '=3', ':-))', ":'-)", ":')", ':*', ':^*', '>:P', ':-P', ':P', 'X-P',
    'x-p', 'xp', 'XP', ':-p', ':p', '=p', ':-b', ':b', '>:)', '>;)', '>:-)',
    '<3'
    ])

# Sad Emoticons
emoticons_sad = set([
    ':L', ':-/', '>:/', ':S', '>:[', ':@', ':-(', ':[', ':-||', '=L', ':<',
    ':-[', ':-<', '=\\', '=/', '>:(', ':(', '>.<', ":'-(", ":'(", ':\\', ':-c',
    ':c', ':{', '>:\\', ';('
    ])

#Emoji patterns
emoji_pattern = re.compile("["
         u"\U0001F600-\U0001F64F"  # emoticons
         u"\U0001F300-\U0001F5FF"  # symbols & pictographs
         u"\U0001F680-\U0001F6FF"  # transport & map symbols
         u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
         u"\U00002702-\U000027B0"
         u"\U000024C2-\U0001F251"
         "]+", flags=re.UNICODE)


#combine sad and happy emoticons
emoticons = emoticons_happy.union(emoticons_sad)

def clean_tweets(tweet):
 
    stop_words = set(stopwords.words('english'))
    word_tokens = word_tokenize(tweet)
    #removing mentions
    tweet = re.sub(r':', '', tweet)
    tweet = re.sub(r'‚Ä¶', '', tweet)
#replace consecutive non-ASCII characters with a space
    tweet = re.sub(r'[^\x00-\x7F]+',' ', tweet)
#remove emojis from tweet
    tweet = emoji_pattern.sub(r'', tweet)
#filter using NLTK library append it to a string
    filtered_tweet = [w for w in word_tokens if not w in stop_words]
    filtered_tweet = []
#looping through conditions
    for w in word_tokens:
#check tokens against stop words , emoticons and punctuations
        if w not in stop_words and w not in emoticons and w not in string.punctuation:
            filtered_tweet.append(w)
    return ' '.join(filtered_tweet)





In [62]:
#replace tabs and spaces

df['text']= df['text'].replace({'\n':" ","\t":" "})

df['text']

292       3/n\nAt 1 million mark top 5 countries with mo...
6704      Sir Umbrellas Are Required to vegetable vendor...
7008      Adding agriculture, mining in the list too.\n#...
7736      #Coronavirus | #Geofencing app will be used to...
13351     Fight against Corona virus. Requesting all to ...
                                ...                        
414315    #CoronavirusOutbreak | #IndiaFightsCorona Let'...
423093    Distributed Ration Kits On Friday 10 April 202...
424383    Distributed Ration Kits On Friday 10 April, at...
426584    Whenever Celebs/ Politicians post a photo of t...
426929    Distributed Ration Kits today at Syed Nagar, N...
Name: text, Length: 23949, dtype: object

In [63]:
#get the hastags

#using regular expressions
hashtags = df['text'].apply(lambda x: re.findall(r'\B#\w*[a-zA-Z]+\w*',x))
print(hashtags)
print(len(hashtags))
df['hashtag'] = hashtags

292                                  [#CoronavirusPandemic]
6704                                         [#Coronavirus]
7008                 [#ReliefPackage, #CoronavirusPandemic]
7736                            [#Coronavirus, #Geofencing]
13351     [#CoronaUpdatesInIndia, #21DaysChallenge, #cor...
                                ...                        
414315    [#CoronavirusOutbreak, #IndiaFightsCorona, #Co...
423093    [#Nampally, #CoronavirusLockdown, #Telangana, ...
424383    [#Nampally, #CoronavirusLockdown, #Telangana, ...
426584    [#Mask, #WearMask, #FaceCover, #Masks, #Corona...
426929    [#Nampally, #CoronavirusLockdown, #Telangana, ...
Name: text, Length: 23949, dtype: object
23949


In [64]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [65]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [0]:
import dask.dataframe as dsk
import multiprocessing as mp
from dask.multiprocessing import get

ddf = dsk.from_pandas(df,npartitions=4*mp.cpu_count())

In [81]:
df['text1'] = df['text'].apply(clean_tweets) #preprocessing using user defined functions
#df['text'] = df['text'].apply(p.clean)
df['text1']= clean_text
df.columns

Index(['status_id', 'user_id', 'screen_name', 'text', 'source', 'is_quote',
       'is_retweet', 'favourites_count', 'retweet_count', 'country_code',
       'place_full_name', 'place_type', 'followers_count', 'friends_count',
       'account_created_at', 'verified', 'lang', 'datetime', 'hashtag',
       'text1'],
      dtype='object')

In [0]:
df['text'] = df['text1']
df = df.drop('text1', axis = 1)

In [83]:
df.text.head()

279    When nation fighting grim battle amidst medica...
292    3/n At 1 million mark top 5 countries recovere...
331    narendramodi ji Is Harsh Decision ke lie We fo...
342    For resorted bursting crackers flying Chinese ...
349    May spirit season new year form invisible host...
Name: text, dtype: object

In [0]:
 #getting sentiment scores and polarity
from textblob import TextBlob

def get_sentiment(text):
    blob = TextBlob(text)
    return [blob.sentiment.polarity, blob.sentiment.subjectivity]




In [85]:

df.head()

,status_id,user_id,screen_name,text,source,is_quote,is_retweet,favourites_count,retweet_count,country_code,place_full_name,place_type,followers_count,friends_count,account_created_at,verified,lang,datetime,hashtag
279,1245501262914674688,113672777,KingKatriar,When nation fighting grim battle amidst medica...,Twitter for Android,True,False,7991,4,IN,"Mumbai, India",city,5360,3340,2010-02-12T16:34:18Z,False,en,2020-04-02 00:00:16+00:00,[#coronavirus]
292,1245863655586979840,857681810830094338,mishralokk,3/n At 1 million mark top 5 countries recovere...,Twitter for Android,False,False,1199,0,IN,"Gurgaon, India",city,2862,235,2017-04-27T19:44:13Z,False,en,2020-04-03 00:00:17+00:00,[#CoronavirusPandemic]
331,1244414125326741504,464935939,asliranjit,narendramodi ji Is Harsh Decision ke lie We fo...,Twitter for Android,False,False,351,0,IN,"New Delhi, India",city,199,37,2012-01-15T19:47:31Z,False,en,2020-03-30 00:00:22+00:00,[#Covid_19]
342,1246950868085325825,102625402,srijanpalsingh,For resorted bursting crackers flying Chinese ...,Twitter for Android,False,False,9518,15,IN,"New Delhi, India",city,31678,2085,2010-01-07T09:11:12Z,True,en,2020-04-06 00:00:28+00:00,[#Covid_19]
349,1249850149498568704,1078338149833555969,dhruvvjyoti,May spirit season new year form invisible host...,Twitter for Android,False,False,30,0,IN,"Dibrugarh, India",city,5,17,2018-12-27T17:13:44Z,False,en,2020-04-14 00:01:11+00:00,"[#SubhoNoboBorsho, #PoilaBaisakh, #StayHomeSta..."


In [92]:
df = pd.DataFrame(df)
polarity,subjectivity = [], []
for text in df.text:
    pol , sub = get_sentiment(text)
    polarity.append(pol)
    subjectivity.append(sub)
    
df['polarity'] = polarity
df['subjectivity'] = subjectivity
df.head()

,status_id,user_id,screen_name,text,source,is_quote,is_retweet,favourites_count,retweet_count,country_code,place_full_name,place_type,followers_count,friends_count,account_created_at,verified,lang,datetime,hashtag,polarity,subjectivity
279,1245501262914674688,113672777,KingKatriar,When nation fighting grim battle amidst medica...,Twitter for Android,True,False,7991,4,IN,"Mumbai, India",city,5360,3340,2010-02-12T16:34:18Z,False,en,2020-04-02 00:00:16+00:00,[#coronavirus],-0.666667,0.666667
292,1245863655586979840,857681810830094338,mishralokk,3/n At 1 million mark top 5 countries recovere...,Twitter for Android,False,False,1199,0,IN,"Gurgaon, India",city,2862,235,2017-04-27T19:44:13Z,False,en,2020-04-03 00:00:17+00:00,[#CoronavirusPandemic],0.500000,0.500000
331,1244414125326741504,464935939,asliranjit,narendramodi ji Is Harsh Decision ke lie We fo...,Twitter for Android,False,False,351,0,IN,"New Delhi, India",city,199,37,2012-01-15T19:47:31Z,False,en,2020-03-30 00:00:22+00:00,[#Covid_19],-0.100000,0.500000
342,1246950868085325825,102625402,srijanpalsingh,For resorted bursting crackers flying Chinese ...,Twitter for Android,False,False,9518,15,IN,"New Delhi, India",city,31678,2085,2010-01-07T09:11:12Z,True,en,2020-04-06 00:00:28+00:00,[#Covid_19],-0.208333,0.225000
349,1249850149498568704,1078338149833555969,dhruvvjyoti,May spirit season new year form invisible host...,Twitter for Android,False,False,30,0,IN,"Dibrugarh, India",city,5,17,2018-12-27T17:13:44Z,False,en,2020-04-14 00:01:11+00:00,"[#SubhoNoboBorsho, #PoilaBaisakh, #StayHomeSta...",0.136364,0.454545


In [94]:
#get the unique hastags

unique_hashtag = set([hash for hashes in df.hashtag for hash in hashes])
print(len(unique_hashtag))

17781


In [0]:
df.to_csv('/content/processed.csv')